In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#pd.read_csv ('/kaggle/input/real-estate-paris/transports.csv')
#pd.read_csv ('/kaggle/input/real-estate-paris/equipements.csv')
#pd.read_csv ('/kaggle/input/real-estate-paris/neighborhood_facilities.csv')

In [3]:
df = pd.read_csv ('/kaggle/input/real-estate-paris/ads.csv')
df

In [4]:
df.columns

In [5]:
df.drop(['date_pub', 'date_update', 'fee'], axis=1, inplace=True, errors='ignore')

**Dataset containe duplicate rows with unique id**

In [6]:
df.describe()

**Finding and removing duplicate rows by adress because we can take address as id for each house**

In [7]:
pd.concat(g for _, g in df.groupby("adress") if len(g) > 1)

**removing duplicate rows by adress**

In [8]:
df = df.drop_duplicates(subset=['adress'],  keep = 'last')

still we can find description column

In [9]:
df.describe()

In [10]:
pd.concat(g for _, g in df.groupby("description") if len(g) > 1)

In [11]:
list(df.loc[df['id'] == "ad_193"]["adress"])

In [12]:
df['district'] = df['adress'].str.extract('(\d+)e', expand=False).str.strip()

In [13]:
df['volume'] = df['adress'].str.extract('(\d+).?m²', expand=False).str.strip()

In [14]:
df['rooms'] = df['adress'].str.extract('(\d+).?[p|P][i|I]', expand=False).str.strip()

In [15]:
df.loc[df['description'].str.contains('(?i)STUDIO') & df['rooms'].isna(), 'rooms'] = 1

In [16]:
df['price'] = df['price'].str.replace(u'\xa0', '').str.replace('€', '').astype(int)

In [17]:
df[df['district'].isna() | df['volume'].isna() | df['rooms'].isna()]

In [18]:
df.loc[1606, 'volume'] = 61
df.loc[1606, 'district'] = 18

In [19]:
df[df['district'].isna() | df['volume'].isna() | df['rooms'].isna()]

In [20]:
#list(df.loc[df['id'] == "ad_2104"]["adress"])

In [21]:
df[["price", "district", "volume", "rooms"]] = df[["price", "district", "volume", "rooms"]].apply(pd.to_numeric)

In [22]:
df.drop(['description', 'adress'], axis=1, inplace=True, errors='ignore')

In [23]:
df

In [24]:
df["rooms"].plot()

In [25]:
df["price"].plot()

In [26]:
df["volume"].plot()

In [27]:
df["rooms"].plot()

In [28]:
pd.value_counts(df['district']).plot.bar()

In [29]:
dist_group = df.groupby(['district']).mean()

In [30]:
dist_group.plot.bar(y=["price"])

In [31]:
dist_group.plot.bar(y=["rooms"])

In [32]:
dist_group.plot.bar(y=["volume"])

In [33]:
df

In [34]:
sns.pairplot(df)

In [35]:
sns.heatmap(df.corr(), annot=True)

In [36]:
df_district = pd.get_dummies(df['district'])
df_district

In [37]:
df.drop(['district'], axis=1, inplace=True, errors='ignore')

In [38]:
df = pd.concat([df, df_district], axis=1)

In [39]:
df.drop(['id'], axis=1, inplace=True, errors='ignore')
X = df.loc[:, df.columns != 'price']
y = df[["price"]]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [41]:
X_train

In [42]:
lm = LinearRegression()

In [43]:
lm.fit(X_train, y_train)

In [44]:
coeff = pd.DataFrame(lm.coef_[0], X.columns, columns=["coefficient"])

In [45]:
coeff

In [46]:
predictions = lm.predict(X_test)

In [47]:
plt.scatter(y_test, predictions)

In [48]:
sns.displot((y_test - predictions), bins=50, kde=True)

In [49]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [50]:
r2_score = lm.score(X_test.values,y_test.values)
print(r2_score*100,'%')